### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value `y` is not in your dataframe.
  
### Question 3

* Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?
  
- `industry`
- `location`
- `lead_source`
- `employment_status`

In [36]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [37]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [38]:
len(df_train),len(df_val),len(df_test)

(876, 293, 293)

In [39]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test= df_test.reset_index(drop=True)

In [40]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [41]:
del df_train['converted']
del df_val['converted']
del df_test['converted']


### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.64
- 0.74
- 0.84
- 0.94

### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model using the same features and parameters as in Q4 (without rounding).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

- `'industry'`
- `'employment_status'`
- `'lead_score'`

> **Note**: The difference doesn't have to be positive.

### Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.

## Homework

> Note: sometimes your answer doesn't match one of the options exactly. 
> That's fine. 
> Select the option that's closest to your solution.


### Dataset

In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv).

Or you can do it with `wget`:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
```

In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not. 

### Data preparation

* Check if the missing values are presented in the features.
* If there are missing values:
    * For caterogiral features, replace them with 'NA'
    * For numerical features, replace with with 0.0 

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer

from IPython.display import display

import matplotlib.pyplot as plt

In [2]:
file = 'course_lead_scoring.csv'
if os.path.exists(file):
    pass
else:
    data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
    !wget $data -O course_lead_scoring.csv  
    

In [3]:
df = pd.read_csv('course_lead_scoring.csv')
df.iloc[:10]

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
5,events,manufacturing,1,59904.0,NaN,africa,6,0.83,1
6,social_media,technology,0,51283.0,NaN,middle_east,2,0.57,0
7,social_media,NaN,5,62975.0,student,europe,4,0.62,1
8,referral,healthcare,4,38648.0,unemployed,south_america,2,0.86,1
9,paid_ads,other,3,59866.0,student,australia,3,0.43,1


In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [7]:
numerical_columns = list(df.dtypes[df.dtypes != 'object'].index)
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [8]:
df[categorical_columns].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [9]:
df[numerical_columns].isnull()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
1457,False,True,False,False,False
1458,False,False,False,False,False
1459,False,False,False,False,False
1460,False,False,False,False,False


In [10]:
df[df[numerical_columns].annual_income.isnull()].index

Index([  14,   58,   70,   72,   79,   96,  106,  127,  131,  133,
       ...
       1412, 1414, 1419, 1424, 1426, 1428, 1432, 1437, 1455, 1457],
      dtype='int64', length=181)

In [11]:
df.iloc[14]

lead_source                    paid_ads
industry                     healthcare
number_of_courses_viewed              3
annual_income                       NaN
employment_status            unemployed
location                    middle_east
interaction_count                     4
lead_score                         0.75
converted                             1
Name: 14, dtype: object

In [12]:
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [13]:
df[categorical_columns].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [14]:
df[numerical_columns] = df[numerical_columns].fillna(0.0)

In [15]:
df[numerical_columns].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [16]:
df.iloc[:10]

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
5,events,manufacturing,1,59904.0,NA,africa,6,0.83,1
6,social_media,technology,0,51283.0,NA,middle_east,2,0.57,0
7,social_media,NA,5,62975.0,student,europe,4,0.62,1
8,referral,healthcare,4,38648.0,unemployed,south_america,2,0.86,1
9,paid_ads,other,3,59866.0,student,australia,3,0.43,1


In [17]:
df[numerical_columns].iloc[14]

number_of_courses_viewed    3.00
annual_income               0.00
interaction_count           4.00
lead_score                  0.75
converted                   1.00
Name: 14, dtype: float64

### Question 1

What is the most frequent observation (mode) for the column `industry`?

- `NA`
- `technology`
- `healthcare`
- `retail`

In [18]:
df.industry.describe()

count       1462
unique         8
top       retail
freq         203
Name: industry, dtype: object

#### The most frequent observation (mode) for column industry is `RETAIL`.

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `interaction_count` and `lead_score`                      0.025393  0.009888
- `number_of_courses_viewed` and `lead_score`               0.009427 -0.004879
- `number_of_courses_viewed` and `interaction_count`       -0.044381 -0.023565
- `annual_income` and `interaction_count`                   0.011959  0.027036	

Only consider the pairs above when answering this question.


In [20]:
categorical = categorical_columns

In [24]:
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [32]:
numerical = ['number_of_courses_viewed','annual_income','interaction_count','lead_score']
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [33]:
df[numerical].corrwith(df.lead_score)

number_of_courses_viewed   -0.004879
annual_income               0.015610
interaction_count           0.009888
lead_score                  1.000000
dtype: float64

In [33]:
df[numerical].corrwith(df.lead_score)

number_of_courses_viewed   -0.004879
annual_income               0.015610
interaction_count           0.009888
lead_score                  1.000000
dtype: float64

In [34]:
df[numerical].corrwith(df.interaction_count)


number_of_courses_viewed   -0.023565
annual_income               0.027036
interaction_count           1.000000
lead_score                  0.009888
dtype: float64

In [43]:
df_full_train[numerical].corrwith(df_full_train.interaction_count)


number_of_courses_viewed   -0.044381
annual_income               0.011959
interaction_count           1.000000
lead_score                  0.025393
dtype: float64

In [59]:
corr_matrix = df[numerical].corr()

In [60]:
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [61]:
max_corr = corr_matrix.unstack().sort_values(ascending=False)
max_corr = max_corr[max_corr < 1]
top_features = max_corr.idxmax()
print(f"The two features with the highest correlation are: {top_features}")

The two features with the highest correlation are: ('annual_income', 'interaction_count')


In [62]:
max_corr

annual_income             interaction_count           0.027036
interaction_count         annual_income               0.027036
lead_score                annual_income               0.015610
annual_income             lead_score                  0.015610
lead_score                interaction_count           0.009888
interaction_count         lead_score                  0.009888
annual_income             number_of_courses_viewed    0.009770
number_of_courses_viewed  annual_income               0.009770
lead_score                number_of_courses_viewed   -0.004879
number_of_courses_viewed  lead_score                 -0.004879
                          interaction_count          -0.023565
interaction_count         number_of_courses_viewed   -0.023565
dtype: float64